In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict , Annotated
from langchain_groq import ChatGroq
import os
from pydantic import BaseModel, Field
import operator   ### For reducer

In [2]:
### Setting up the LLM
model=ChatGroq(model="llama-3.1-8b-instant",
                api_key=os.getenv("API_KEY"))

In [7]:
### From LLM model AT the first  Phase we arre expecting two things:
### 1. Text feedback on the essay
### 2. A numerical score out of 10
### each node at the first phase of evaluation should return both these things

class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    Score: int = Field(description="Numerical score out of 10",ge=0,le=10)


In [8]:
structured_model = model.with_structured_output(EvaluationSchema)

In [9]:
essay="""Artificial Intelligence (AI) in India is emerging as a transformative force, reshaping industries, governance, and everyday life. AI refers to the ability of machines to perform tasks that typically require human intelligence—such as reasoning, learning, and decision-making. In recent years, India has recognized the enormous potential of AI to drive social and economic growth. The government, through initiatives like **NITI Aayog’s “National Strategy for Artificial Intelligence – AI for All”**, has emphasized using AI to address key national challenges in sectors like healthcare, agriculture, education, transportation, and smart cities. This vision aims not only to promote innovation but also to ensure that the benefits of AI reach every section of society, especially in rural and underdeveloped regions.

The growth of AI in India has been driven by both public and private sectors. Major Indian IT companies such as **Infosys, Wipro, and TCS**, along with numerous startups like **Haptik, Mad Street Den, and SigTuple**, are actively developing AI-based solutions. Additionally, global tech giants like **Google, Microsoft, Amazon, and IBM** have set up AI research labs and innovation centers in India, recognizing the country’s immense potential and talent pool. AI applications are already visible in various fields: in **healthcare**, AI systems assist in diagnosing diseases like tuberculosis and cancer through image analysis; in **agriculture**, AI tools help farmers predict weather patterns, optimize crop yields, and detect pests; in **education**, personalized learning platforms powered by AI enhance student engagement and performance; in **governance**, AI-driven data analytics supports better policy-making, fraud detection, and citizen service delivery.

Despite its promising growth, the adoption of AI in India faces several challenges. One of the most pressing issues is the **shortage of skilled AI professionals**. While India produces a large number of engineering graduates every year, only a small fraction are trained in advanced technologies like AI and machine learning. Another concern is **data privacy and security**, as AI systems rely on massive amounts of data to function effectively. Ensuring the protection of citizens’ personal information while promoting innovation is a delicate balance. Moreover, India’s **digital divide**—the gap between urban and rural access to technology—poses a significant barrier to inclusive AI development. Ethical questions about bias in AI algorithms and potential job losses due to automation also need to be addressed through careful planning and regulation.

To tackle these issues, the Government of India has launched multiple initiatives. The **IndiaAI Mission**, **Digital India**, and **Centers of Excellence in AI** have been established to promote research, innovation, and skill development in this field. Academic institutions are introducing AI-focused courses, and collaborations between government, academia, and industry are strengthening the AI ecosystem. According to several economic studies, AI could contribute **up to $1 trillion to India’s GDP by 2035**, making it a cornerstone of the country’s digital economy. With over a billion people, a rapidly growing internet user base, and a young, tech-savvy workforce, India is uniquely positioned to become a global hub for AI innovation.

In conclusion, Artificial Intelligence represents a tremendous opportunity for India to enhance productivity, improve governance, and foster inclusive growth. However, realizing this potential requires sustained investment in education, digital infrastructure, and ethical governance frameworks. If implemented responsibly, AI can bridge social and economic gaps, modernize industries, and elevate India’s position on the global technology map. As India continues its journey into the digital age, AI will undoubtedly play a pivotal role in shaping a smarter, more efficient, and equitable future for all.
"""

In [10]:
prompt=f'Evaluate the language quality of following essay and provide detailed feedback along with a score out of 10.\n\nEssay:\n{essay}'
structured_result=structured_model.invoke(prompt)
result_dict=structured_result.dict()
print(result_dict.get('feedback'))
print(f"Score out of 10: {result_dict.get('score', 0)}")

The essay is well-structured and effectively conveys the importance of Artificial Intelligence in India. It provides a comprehensive overview of the current state of AI in the country, its applications, and the challenges it faces. The writer also suggests potential solutions to overcome these challenges. The use of specific examples, such as the National Strategy for Artificial Intelligence and the IndiaAI Mission, adds credibility to the essay. However, the writer could have provided more nuanced analysis of the benefits and drawbacks of AI and explored alternative perspectives on the issue. Nevertheless, the essay demonstrates a clear understanding of the topic and is well-written, making it a strong candidate for a high score. Some minor suggestions for improvement include using more precise language in the introduction and providing more concrete data to support the claim that AI could contribute up to $1 trillion to India’s GDP by 2035.
Score out of 10: 0


C:\Users\Yashraj Sharma\AppData\Local\Temp\ipykernel_22720\2581020349.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_dict=structured_result.dict()


In [11]:
class UPSCState(TypedDict):

    essay:str
    language_feedback:str
    analytical_feedback:str
    clarity_feedback:str
    overall_feedback:str
    individual_scores:Annotated[list[int],operator.add] ##To prevent overriding during reduction
    average_score:float

In [13]:
def language_evaluation(state:UPSCState):
    prompt=f'Evaluate the language quality of following essay and provide detailed feedback along with a score out of 10.\n\nEssay:\n{state["essay"]}'
    structured_result=structured_model.invoke(prompt)
    result_dict=structured_result.dict()

    language_feedback=result_dict.get('feedback')
    language_score=result_dict.get('Score')

    return {'language_feedback':language_feedback,
            'individual_scores':[language_score]}



In [14]:
def analytical_evaluation(state:UPSCState):
    prompt=f'Evaluate the analytical quality of following essay and provide detailed feedback along with a score out of 10.\n\nEssay:\n{state["essay"]}'
    structured_result=structured_model.invoke(prompt)
    result_dict=structured_result.dict()


    analytical_feedback=result_dict.get('feedback')
    analytical_score=result_dict.get('Score')

    return {'analytical_feedback':analytical_feedback,
            'individual_scores':[analytical_score]}




In [15]:
def clarity_evaluation(state:UPSCState):
    prompt=f'Evaluate the clarity of ideas in following essay and provide detailed feedback along with a score out of 10.\n\nEssay:\n{state["essay"]}'
    structured_result=structured_model.invoke(prompt)
    result_dict=structured_result.dict()

    clarity_feedback=result_dict.get('feedback')
    clarity_score=result_dict.get('Score')

    return {'clarity_feedback':clarity_feedback,
            'individual_scores':[clarity_score]}


In [16]:
def overall_evaluation(state:UPSCState):
    language_feedback=state['language_feedback']
    analytical_feedback=state['analytical_feedback']
    clarity_feedback=state['clarity_feedback']
    prompt=f'Provide overall feedback for the following essay based on {language_feedback}, {analytical_feedback}, and {clarity_feedback}.\n\nEssay:\n{state["essay"]}'
    overall_feedback=model.invoke(prompt).content.strip()
 
    
    average_score=sum(state['individual_scores'])/len(state['individual_scores']) if state['individual_scores'] else 0
    state['average_score']=average_score

    state['overall_feedback']=overall_feedback

    return {'overall_feedback':overall_feedback,
            'average_score':average_score}

In [17]:
### Defining Graph:
graph=StateGraph(state_schema=UPSCState)

### Defining Nodes for language, analytical and clarity evaluation in parallel
graph.add_node(
    name="language_evaluation",
    node=language_evaluation
)
graph.add_node(
    name="analytical_evaluation",
    node=analytical_evaluation
)
graph.add_node(
    name="clarity_evaluation",
    node=clarity_evaluation
)

graph.add_node(
    name="overall_evaluation",
    node=overall_evaluation
    )

### Defining Edges:
graph.add_edge(START,"language_evaluation")
graph.add_edge(START,"analytical_evaluation")
graph.add_edge(START,"clarity_evaluation")
graph.add_edge("language_evaluation","overall_evaluation")
graph.add_edge("analytical_evaluation","overall_evaluation")
graph.add_edge("clarity_evaluation","overall_evaluation")
graph.add_edge("overall_evaluation",END)

### Executing the graph:
workflow=graph.compile()

In [22]:
initial_state={
    'essay':essay,
}

final_state=workflow.invoke(initial_state)
print("Final Evaluation Results:")
print(f"Overall Feedback: {final_state['overall_feedback']}")
print(f"Average Score out of 10: {final_state['average_score']}")


C:\Users\Yashraj Sharma\AppData\Local\Temp\ipykernel_22720\481186363.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_dict=structured_result.dict()


Final Evaluation Results:
Overall Feedback: The essay provides a comprehensive and informative overview of Artificial Intelligence in India, discussing its growth, applications, challenges, and government initiatives. Here are some overall feedback and suggestions for improvement:

**Strengths:**

1. The essay is well-structured and easy to follow.
2. The language is clear, concise, and engaging.
3. The author demonstrates a good command of vocabulary and grammar.
4. The essay presents a comprehensive and optimistic view of AI's potential in India.
5. The author's enthusiasm and passion for the topic are evident throughout.

**Weaknesses:**

1. The essay would benefit from more specific examples and data to support the claims.
2. A more nuanced discussion of the challenges and limitations of AI in India is needed.
3. The potential risks and downsides of AI are not thoroughly explored.
4. Some statements, such as "AI could contribute up to $1 trillion to India’s GDP by 2035," would bene